# User-based Collaborative Filtering using Pandas

In this notebook I explore a technique to recommend items, called __collaborative filtering__.

The basic idea of user-based collaborative filtering for recommender systems is the following:
The task is to recommend items to a given user x. 

Which items do we recommend? First, we find N other users in the neighborhood of user x. Then, we predict the rating of unrated items by user x by using the ratings of the N other users. The recommended items must not be rated yet by user X. 

I use the __MovieLens 100K Dataset__ (https://grouplens.org/datasets/movielens/). Copy the dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip, and unzip it into a folder.

2 files of the dataset are used:
- `ml-100k/u.data` contains the 100k ratings.
- `ml-100k/u.item` provides a mapping from a movie id to the name of the movie.

In [1]:
# data manipulation
import numpy as np
import pandas as pd

In [2]:
# paths to the files. download them from http://files.grouplens.org/datasets/movielens/ml-100k.zip if you don't have them yet
MOVIE_RATINGS_PATH = 'ml-100k/u.data'
MOVIE_NAMES_PATH = 'ml-100k/u.item'

# number of neighbors of user
N_NEIGHBORS = 10

# number of recommendations
N_RECOMMENDATIONS = 5

# Reading the Ratings Data

In [3]:
def read_ratings(path_to_ratings):
    """
    Read the raw data of the movie ratings.
    
    Returns a list of tuples:
    (user id, movie id, rating)
    """

    data = []
    with open(path_to_ratings) as f:
        for line in f:
            # user id | item id | rating | timestamp
            pieces = line.split()
            user_id = int(pieces[0])
            movie_id = int(pieces[1])
            rating = float(pieces[2])
            data.append((user_id, movie_id, rating))
        
    return data

In [4]:
def read_names(path_to_names):
    """
    Read the mapping of movie id -> movie name
    
    Returns a dictionary
    {movie id -> movie name}
    """

    data = {}
    with open(path_to_names) as f:
        for line in f:
            # movie id | movie title | ...
            pieces = line.split('|')
            movie_id = int(pieces[0])
            title = pieces[1]
            data[movie_id] = title
        
    return data

In [5]:
ratings = read_ratings(MOVIE_RATINGS_PATH)
ratings = pd.DataFrame(data=ratings, columns=['user', 'movie', 'rating'])#.sample(100, random_state=42)
ratings = ratings.astype(int)
ratings.head()

,user,movie,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [6]:
# number of unique movies
ratings.movie.nunique()

1682

In [7]:
# number of unique users
ratings.user.nunique()

943

# Pick a random sample for testing purposes

In [8]:
sample = ratings.sample(random_state=42)
user_id = sample.user.values[0]
item_id = sample.movie.values[0]

In [9]:
user_id, item_id

(877, 381)

# Transforming the user rating data

The Movielens dataset comes in the so-called _long format_. The dataset is structured as tuples of

`user id, movie id, rating`

That means, multiple rows may have the same user_id. One row for every movie that the user rated.

In user-based collaborative filtering, we need the data in the long format.

Therefore, we conver from long to wide format.

In [10]:
# convert long to wide
ratings_raw = ratings.copy()
ratings = ratings.pivot(index='user', columns='movie', values='rating')

In [11]:
ratings.shape

(943, 1682)

In [12]:
ratings.head()

movie,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# select user, movie
ratings.loc[user_id, item_id]

4.0

In [14]:
# ratings vector of specific user
ratings.loc[user_id, :].sample(10)

movie
495    NaN
628    NaN
1544   NaN
5      NaN
918    NaN
1635   NaN
1479   NaN
1039   NaN
53     NaN
1381   NaN
Name: 877, dtype: float64

# Measuring similarity between users

After converting the dataset into the wide format, with rows as ratings vector of every user.

The ratings vectors are used to measure the similarity or dissimilarity between users. When 2 users have given the same movies the same ratings, we can say that the 2 users are similar with respect to their movie taste. Vice versa, when 2 users have rated the same movie very differently, the users have very different taste in movies.

User-based collaborative filtering focuses on finding users with very similar taste in movies to recommend movies to someone.

Here, we use the Pearson correlation to measure similarity between 2 ratings vectors (= 2 users). Pearson correlation has a value between -1 and +1. 0 means no correlation. +1 means very similar. -1 means very dissimilar.

We compute the similarity of a given user with all the other users in the dataset.

In [15]:
def pearson_similarity(v1, v2):
    """
    Compute the Pearson correlation between to ratings vectors.
    
    pd.corr() function can handle missing data.
    
    parameters: 
    - v1, v2: pd.Series, ratings vectors
    
    returns:
    - float
    
    """
    
    pearson = v1.corr(v2)
    
    return pearson

In [16]:
# let us test
pearson_similarity(
    pd.Series([1, 2, 3, 4, 5, np.NaN]), 
    pd.Series([2, 1, 4, 5, np.NaN, 1])
)

0.84852813742385713

In [17]:
def compute_similarities(user_id, ratings_matrix):
    """
    Compute the similarity of a given user with all the other users in the dataset.
    
    Remove the similarity value for the given user from the result.
    
    returns:
        - a pd.Series with the user id's as index, and similarity as series values
    """
    
    # get ratings of user to re-use in the similarity computation
    ratings_user = ratings_matrix.loc[user_id,:]
    
    # calculate the similarity between the given user and the other users
    similarities = ratings_matrix.apply(
        lambda row: pearson_similarity(ratings_user, row), 
        axis=1)

    similarities = similarities.to_frame(name='similarity')

    # find most similar users to the given user
    similarities = similarities.sort_values(by='similarity', ascending=False)
    
    # drop the similarity of the user (should be ~1 anyways)
    similarities = similarities.drop(user_id)
    
    return similarities

In [18]:
# let us test
similarities = compute_similarities(user_id, ratings)
similarities.head()

F:\Program_Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
F:\Program_Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2496: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


,similarity
user,
917,1.0
300,1.0
386,1.0
789,1.0
869,1.0


# Predicting Movie Ratings

The user similarities allow to predict the rating that a user would give to an unrated movie. The recipe to make a rating prediction goes as follows:
- find the N most similar neighbors of the user, who have rated the given movie
- average the rating of that movie of the N neighbors of the user

In [19]:
def predict_rating(item_id, ratings, similarities, N=10):
    """
    Predict the rating of a given item by a user, given the ratings of similar users.
    Takes the N users with the highest similarity measure, AND who have rated the given item.
    Returns the average rating of the most similar users who previously rated the item.
    
    parameters:
    - item_id: int, item that needs a rating prediction
    - ratings: pd.DataFrame
    - similarities: pd.DataFrame
    - N: int, number of neighbors to use for rating prediction
    
    returns:
    - a float representing the predicted rating for the given item
    
    """
    
    # get the ratings of all users for the specific item
    users_ratings = ratings.loc[:, item_id]
    
    # only keep users who rated the given item, otherwise you won't be able to generate a prediction based on the users ratings
    most_similar_users_who_rated_item = similarities.loc[~users_ratings.isnull()]
    
    # keep N users with highest similarities to given user who also rated the given item
    N_most_similar_users = most_similar_users_who_rated_item.head(N)
    
    # find ratings item for most similar users:
    ratings_for_item = ratings.loc[N_most_similar_users.index, item_id]
    
    # predict the rating of the item by averaging the ratings of that item of the most similar users
    return ratings_for_item.mean()

In [20]:
# let us test
predict_rating(item_id, ratings, similarities, N=N_NEIGHBORS)

4.0

# Recommending movies

Once we can predict what rating a user will give to a given movie, recommending movies to that user becomes straight forward.

We recommend movies as follows:
- find the movies that the user has not rated.
- predict the rating that the user would give to these unrated movies.
- find the N movies with the highest ratings.

In [21]:
movie_names = read_names(MOVIE_NAMES_PATH)

In [22]:
def recommend(user_id, ratings, movie_names, n_neighbors=10, n_recomm=5):
    """
    
    Recommend N movies for a given user based on ratings data.
    
    1. get the ratings of the user
    2. get the movies that the user has not rated
    3. compute the similarities between the user and the other users
    4. generate movie ratings predictions for the user based on the similarities with other users
    5. find the N movies with the highest predicted ratings
    
    parameters:
    - user_id: int, user to generate recommendations for
    - ratings: pd.DataFrame, user-movie ratings
    - movie_names: dict, mapping of (movie id -> movie name)
    - n_neighbors: int: the number of neighbors to use to generate rating predictions
    - n_recomm: int, number of movies to recommend
    
    returns:
    - pd.DataFrame with [movie_id, rating, movie name]
    
    """
    
    # get the ratings of the user
    ratings_user = ratings.loc[user_id, :]
    
    # all the items a user has not rated, that can be recommended
    all_items = ratings.loc[user_id,:]
    unrated_items = all_items.loc[all_items.isnull()]
    
    # convert the index with item ids into Series values
    unrated_items = unrated_items.index.to_series(name='item_ids').reset_index(drop=True)
    print('User {} has {} unrated items.'.format(user_id, len(unrated_items)))
    
    # compute user similarities
    similarities = compute_similarities(user_id, ratings)
        
    # generate predictions for unseen items based on the user similarity data
    predictions = unrated_items.apply(lambda d: predict_rating(d, ratings, similarities, N=n_neighbors))
    
    # sort items by highest predicted rating
    predictions = predictions.sort_values(ascending=False)
    
    # recommend top N items
    recommends = predictions.head(n_recomm)
    
    # reformat the result
    recommends = recommends.to_frame(name='predicted_rating')
    recommends = recommends.rename_axis('movie_id')
    recommends = recommends.reset_index()
    
    recommends['name'] = recommends.movie_id.apply(lambda d: movie_names[d])
    
    return recommends

In [23]:
# let us test
recommends = recommend(user_id, ratings, movie_names, n_neighbors=N_NEIGHBORS, n_recomm=N_RECOMMENDATIONS)
recommends

F:\Program_Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
F:\Program_Files\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2496: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


User 877 has 1601 unrated items.


,movie_id,predicted_rating,name
0,737,5.0,Sirens (1994)
1,1041,5.0,Forget Paris (1995)
2,1454,5.0,Angel and the Badman (1947)
3,1418,5.0,"Joy Luck Club, The (1993)"
4,1385,5.0,Roseanna's Grave (For Roseanna) (1997)
